In [1]:
import os
import pandas as pd
import warnings
import funciones_tpm as fc
import numpy as np

In [2]:
#Ocultar advertencias/warnings mas no los borra
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [3]:
#Año evaluado
Añoeval = 2025

**Variables globales de las rutas necesarias**

In [72]:
#Rutas del archivo de la COOISPI
Ruta_COOISPI = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\COOISPI\Datos_COOISPI_Mezclas.csv"                          # Ruta COOISPI en csv

#Rutas del archivo del EGE
Ruta_Archivo_EGE = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Backup_BD_Salones\Datos_Mezclas_2021.csv"               # Ruta Mezclas

#Rutas del archivo de Novedades
Ruta_Novedades = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Consolidados_Salones\Mezclas\Mezclas_Novedades_V2.xlsx"   # Ruta del archivo de Novedades

#Rutas del archivo de Consolidados
Ruta_Consolidados = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Consolidados_Salones\Mezclas\Consolidado_V2.xlsx"      # Ruta del archivo de Consolidados

#Ruta de salida
# Ruta_Mezclas = r"\\10.28.5.232\s3-1colcafeci-servicios-jtc\TPM\Colcafé Formularios\BD Sobrepeso\Consolidados_Salones\Mezclas"                             # DB_Mezclas_Out_vf.csv

#Validacion de existencia de las rutas
for nombre, ruta in {
  "Ruta_COOISPI": Ruta_COOISPI,
  "Ruta_Archivo_EGE": Ruta_Archivo_EGE,
  "Ruta_Novedades": Ruta_Novedades,
  "Ruta_Consolidados": Ruta_Consolidados,
  # "Ruta_Mezclas": Ruta_Mezclas,
}.items():
  if not os.path.exists(ruta):
    raise FileNotFoundError(f"La ruta '{nombre}' no fue encontrada.")
  else:
    print(f"Ruta '{nombre}' encontrada.")

Ruta 'Ruta_COOISPI' encontrada.
Ruta 'Ruta_Archivo_EGE' encontrada.
Ruta 'Ruta_Novedades' encontrada.
Ruta 'Ruta_Consolidados' encontrada.


In [73]:
#Leer_archivo COOISPI csv
df_COOISPI_COMB = fc.leer_archivo(Ruta_COOISPI)

In [74]:
#Convertir columna 'Codigo' a str para futuros procesor (merge y tabla pivote)
df_COOISPI_COMB['Codigo'] = df_COOISPI_COMB['Codigo'].astype(str)

In [75]:
#Funcion para leer archivo el EGE/TPM
df_Mezclas = fc.leer_archivo(Ruta_Archivo_EGE, 'Datos')

In [76]:
df_Mezclas.head()

,Número,Día:,Mes:,Año:,Semana:,Turno:,Hora de Inicio:,Hora Final:,Máquina / Equipo:,Código y Descrip. / Producto:,...,Legal Contractual\nPeso min,Legal Contractual\nMarcación min,Legal Contractual\nOtros min,Legal Contractual\nPeso Ev,Legal Contractual\nMarcación Ev,Legal Contractual\nOtros Ev,Reprocesables\nOperativos,Reprocesables\nQA,No Reprocesabales\nPCC Contaminación,No Reprocesables\nInnocuidad
0,57517,4,Enero,2021,1,1,11:10:00,13:15:00,TOYO,1014138 Coffee Crem COLCAFE 3g 24un 30ple,...,NaN,NaN,NaN,NaN,NaN,NaN,30.0,80.0,NaN,NaN
1,57518,4,Enero,2021,1,2,13:35:00,21:35:00,TOYO,1014138 Coffee Crem COLCAFE 3g 24un 30ple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN
2,57519,5,Enero,2021,1,2,13:40:00,21:20:00,TOYO,1038938 Cafe COLCAFE clasico 1.5g 48 sob 30 pleg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198.0,NaN,NaN
3,57520,6,Enero,2021,1,1,07:35:00,12:40:00,TOYO,1038938 Cafe COLCAFE clasico 1.5g 48 sob 30 pleg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57521,4,Enero,2021,1,1,05:35:00,13:15:00,TOYO 2,1038938 Cafe COLCAFE clasico 1.5g 48sob 30ple,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.0,NaN,NaN


In [77]:
#Reemplazar el nombre de la columna "Día:" y "Mes_N" para estandarizar los scripts
df_Mezclas.rename(columns={"Día:":"Dia", "Mes_N":"IdMes"}, inplace=True)

In [78]:
#Eliminar_filas_vacias
numeros_vacios_filtrados = fc.eliminar_filas_vacias(df_Mezclas, 'Número')

In [79]:
#Crear_fecha
df_Mezclas = fc.crear_fecha(df_Mezclas)

In [80]:
#Tomaré solo las columnas de interes del informe de mezclas para el sobrepeso
columnas=['Número','Dia','Fecha','Mes:', 'IdMes', 'Año:', 'Máquina / Equipo:', 'Semana:', 'Turno:', 'Código y Descrip. / Producto:', 'Unidades Producidas (Conformes) :', 'Peso Promedio de la unidad (K):','Gramaje (K):']
df_Mezclas = df_Mezclas[columnas]

In [ ]:
# df_Mezclas[df_Mezclas['Número'] == 84813]

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,Unidades Producidas (Conformes) :,Peso Promedio de la unidad (K):,Gramaje (K):
27255,84813,7,2025-08-07,Agosto,8,2025,INGEPACK,32,2,1003690 Cafe COLCAFE polvo ST1 Cj25tirilX24X10g,168/00,0.01006,0.01


In [81]:
df_Mezclas.dtypes

Número                                 int64
Dia                                    int64
Fecha                                 object
Mes:                                  object
IdMes                                  int64
Año:                                   int64
Máquina / Equipo:                     object
Semana:                               object
Turno:                                 int64
Código y Descrip. / Producto:         object
Unidades Producidas (Conformes) :     object
Peso Promedio de la unidad (K):       object
Gramaje (K):                         float64
dtype: object

In [82]:
#Filtrar_desde_anio
df_Mezclas = fc.filtrar_desde_anio(df_Mezclas, 2024)

In [83]:
#Especificar las columnas en las que deseas aplicar el filtro para eliminar los valores nulos de las columnas de Gramaje, Unidades producidas y Peso promedio
columnas_filtrar_nulos = ['Gramaje (K):', 'Unidades Producidas (Conformes) :', 'Peso Promedio de la unidad (K):']

In [84]:
#Eliminar_valores_nulos
df_Mezclas_nan_ceros, df_Mezclas = fc.filtrar_nulos(df_Mezclas, columnas_filtrar_nulos)

In [85]:
df_Mezclas.dtypes

Número                                 int64
Dia                                    int64
Fecha                                 object
Mes:                                  object
IdMes                                  int64
Año:                                   int64
Máquina / Equipo:                     object
Semana:                               object
Turno:                                 int64
Código y Descrip. / Producto:         object
Unidades Producidas (Conformes) :     object
Peso Promedio de la unidad (K):       object
Gramaje (K):                         float64
dtype: object

In [86]:
df_Mezclas.head()

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,Unidades Producidas (Conformes) :,Peso Promedio de la unidad (K):,Gramaje (K):
16991,74528,2,2024-01-02,Enero,1,2024,ENFLEX,1,1,1014168 Polvo st1 10g sobre ind. Horiz Ecu.,3000.0,0.0101,0.0100
16992,74529,2,2024-01-02,Enero,1,2024,INGEPACK 2,1,1,1057776 Cafe Colcafe Clasico10g25sobres,19500.0,0.01005,0.0100
16993,74531,2,2024-01-02,Enero,1,2024,TOYO,1,3,1056678 Cafe COLCAFE clasico 1.5g 48sob 30ple,106560.0,0.00151,0.0015
16994,74532,2,2024-01-02,Enero,1,2024,ROVEMA,1,3,1057775 Cafe COLCAFE clasico 10g10sob12tarj,27240.0,0.01009,0.0100
16995,74533,2,2024-01-02,Enero,1,2024,INGEPACK,1,3,1003690 Cafe COLCAFE polvo ST1 Cj25tirilX24X10g,25800.0,0.01007,0.0100


**Modificación de los tipos de datos del archivo de TPM de Mezclas**

In [ ]:
#Validar_y_convertir_datos
df_non_conver, df_Mezclas = fc.validar_numericos(df_Mezclas, 'Peso Promedio de la unidad (K):')

df_non_unidades, df_Mezclas = fc.validar_numericos(df_Mezclas, 'Unidades Producidas (Conformes) :')

In [55]:
df_Mezclas.dtypes

Número                                 int64
Dia                                    int64
Fecha                                 object
Mes:                                  object
IdMes                                  int64
Año:                                   int64
Máquina / Equipo:                     object
Semana:                               object
Turno:                                 int64
Código y Descrip. / Producto:         object
Unidades Producidas (Conformes) :     object
Peso Promedio de la unidad (K):      float64
Gramaje (K):                         float64
dtype: object

In [88]:
84813
df_Mezclas[df_Mezclas['Número'] == 84813].head()

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,Unidades Producidas (Conformes) :,Peso Promedio de la unidad (K):,Gramaje (K):


In [89]:
df_Mezclas[df_Mezclas['Unidades Producidas (Conformes) :'] == np.nan]

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,Unidades Producidas (Conformes) :,Peso Promedio de la unidad (K):,Gramaje (K):


In [90]:
df_Mezclas[df_Mezclas['Unidades Producidas (Conformes) :'] == 0]

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,Unidades Producidas (Conformes) :,Peso Promedio de la unidad (K):,Gramaje (K):
25153,82707,2,2025-04-02,Abril,4,2025,TOYO 7,14,2,1072097 Cappuccino COLCAFE Avellana 6x18gx12 ECU,0.0,0.0180,0.018
25691,83247,7,2025-05-07,Mayo,5,2025,TOYO 7,19,3,1071324 Cappuccino COLCAFE Mocca 12x6x18G ESP,0.0,0.0180,0.018
26574,84132,28,2025-06-28,Junio,6,2025,INGEPACK,26,3,1064334 Café Colcafe Intnso 12x10g CAM,0.0,0.0100,0.010
26575,84133,28,2025-06-28,Junio,6,2025,ROVEMA,26,3,1057775 Cafe COLCAFE clasico 10g10sob12tarj,0.0,0.0101,0.010


In [91]:
#Extraer_codigo
df_Mezclas = fc.extraer_codigo(df_Mezclas)

In [81]:
#Convertir a numerico sino asignar NaN
df_Mezclas['Gramaje (K):'] = pd.to_numeric(df_Mezclas['Gramaje (K):'], errors='coerce')

In [82]:
#Eliminar_codigos_nan
df_Mezclas, df_Mezclas_Codigos_nan = fc.eliminar_codigos_nan(df_Mezclas)

In [ ]:
#Calcular_columnas
df_Mezclas = fc.calcular_columnas(df_Mezclas)

**Generación de Novedades**

In [109]:
Sobrepeso_Novedades = 0.05 #5% de Sobrepeso

In [110]:
#Generar_novedades
df_Mezclas_Nov_Sobrepeso = fc.generar_novedades(df_Mezclas, Sobrepeso_Novedades)

In [111]:
#Agregar columna 'Costo/kg'
df_Mezclas['Costo/kg'] = 0.0

**Tabla de Consolidado Mensual**

In [112]:
#Pivote_consolidado
df_Mezclas_Mes = fc.pivote_consolidado(df_Mezclas)

In [113]:
#Tomar datos desde el año 2024 en adelante
df_Mezclas_Mes = fc.filtrar_desde_anio(df_Mezclas_Mes, 2024)

**Creación de la agrupación Mensual del archivo de semielaborados**

In [114]:
df_costo_semi = df_COOISPI_COMB.copy(deep=True)

In [115]:
#Creacion de la agrupacion mensual del archivo semielaborado
df_costo_semi_Mes = fc.pivote_semielaborados(df_costo_semi)

**Generación del nuevo archivo consolidado V2**

In [116]:
df_Mezclas_2 = df_Mezclas.copy(deep=True)

In [117]:
#Traer la columna 'Costo/kg' al dataframe original
df_Mezclas_2 = fc.merge_costo(df_Mezclas_2, df_costo_semi_Mes)

In [118]:
#Eliminar la columna Costo/kg antigua y renombrar la nueva
df_Mezclas_2 = fc.modificar_costo(df_Mezclas_2)

In [119]:
#Definir meta
df_Mezclas_2['Meta'] = df_Mezclas_2['Máquina / Equipo:'].map({'ENFLEX': 0.0056,
                                                              'TRANSPACK 2': 0.0033,
                                                              'INGEPACK': 0.0072,
                                                              'ROVEMA':0.0078,
                                                              'LANIC':0.0055,
                                                              'TOYO 7':0.0065,
                                                              'TOYO':0.0088,
                                                              'ENCAPSULADORA':0.006,
                                                              'INGEPACK 2':0.0068})

In [120]:
#Calcular columna 'Ahorros/Perdidas'
df_Mezclas_2 = fc.calcular_ahorros_perdidas(df_Mezclas_2)
# df_Mezclas_2.head()

In [121]:
#Eliminar registro con Número = 81187
df_Mezclas_2.drop(df_Mezclas_2.loc[df_Mezclas_2['Número'] == 81187].index, inplace=True)

**Unión de los dataframes en un dataframe totalizado**

In [122]:
df_totalizado_Mes = df_Mezclas_Mes.copy(deep=True)

In [123]:
#Validar que no hay filas repetidas en df_totalizado_Mes
fc.filas_repetidas(df_totalizado_Mes)

No hay filas duplicadas.


In [124]:
#Fusionar los DataFrames en base a 'Año:', 'Codigo' y 'Mes:'
df_totalizado_Mes = fc.unir_dataframes(df_totalizado_Mes, df_costo_semi_Mes)

In [125]:
#Metodo diferente a los demas calcular_columnas_totalizado ya que sus metas de sobrepeso no son fijas
df_totalizado_Mes = fc.calcular_columnas_totalizado_mezclas(df_totalizado_Mes)
# df_totalizado_Mes.head()

In [126]:
# Obtener Codigos unicos de df_Mezclas
df_Codigos_Mezclas = fc.eliminar_duplicados_columna(df_Mezclas, 'Codigo')

In [127]:
# Obtener Maquinas únicas de TPM o df_Codigos_Mezclas
df_Maquinas_Mezclas = fc.eliminar_duplicados_columna(df_Mezclas, 'Máquina / Equipo:')

In [128]:
# Obtener Codigos únicos de COOISPI
df_Codigos_Mezclas_COOISPI = fc.eliminar_duplicados_columna(df_COOISPI_COMB, 'Codigo')

**Codigos que están en TPM pero no están en la COOISPI**

In [129]:
#Obtener Codigos que están en TPM (df_Codigos_Mezclas) pero NO en COOISPI(df_Codigos_Mezclas_COOISPI)
df_Mezclas_Codigos_unicos = df_Codigos_Mezclas[~df_Codigos_Mezclas['Codigo'].isin(df_Codigos_Mezclas_COOISPI['Codigo'])]

**Generación de un archivo de fechas Automáticas**

In [130]:
#Generar dataframe con fechas dese 2024-01-01 hasta el dia actual
df_Fechas = fc.generar_fechas("2024-01-01")

In [131]:
# print(df_Mezclas.shape)
# print(df_COOISPI_COMB.shape)
# print(df_Mezclas_Mes.shape)
# print(df_Codigos_Mezclas.shape)
# print(df_Maquinas_Mezclas.shape)
# print(df_totalizado_Mes.shape)
# print(df_Fechas.shape)
# print(df_Mezclas_2.shape)
# print()
# print(df_Mezclas_Nov_Sobrepeso.shape)
# print(df_Mezclas_Codigos_nan.shape)
# print(df_Mezclas_Codigos_unicos.shape)
# print(df_Mezclas_nan_ceros.shape)
# print(df_non_conver.shape)

**Consolidado csv**

In [132]:
# Ruta para consolidado df_Mezclas_2
# df_Mezclas_2.to_csv(Ruta_Mezclas + "DB_Mezclas_Out_vf.csv", index=False)

**Archivo de consolidado**

In [133]:
#Diccionario de hojas para el archivo de Consolidado_V2
dataframes_consolidado = {
    "Hoja1": df_Mezclas,
    "Hoja2": df_COOISPI_COMB,
    "Hoja3": df_Mezclas_Mes,
    "Hoja4": df_Codigos_Mezclas,
    "Hoja5": df_Maquinas_Mezclas,
    "Hoja6": df_totalizado_Mes,
    "Hoja7": df_Fechas,
    "Hoja8": df_Mezclas_2,
}
#Actualizar hojas del excel de Consolidado_V2
fc.actualizar_hojas_excel(Ruta_Consolidados, dataframes_consolidado)

Procesando Consolidado...
✅ Las hojas en el Archivo de Consolidado han sido limpiadas y actualizadas correctamente.


**Archivo de novedades: Este archivo compila las diferentes novedades presentes en los archivos de TPM**

In [ ]:
#Diccionario de hojas para el archivo de novedades
dataframes_novedades = {
    "Sobrepeso mayor  5%": df_Mezclas_Nov_Sobrepeso,           #Novedades de sobrepeso
    "Codigos Nulo TPM": df_Mezclas_Codigos_nan,                #Novedades codigos nulos
    "Codigos unicos en TPM": df_Mezclas_Codigos_unicos,        #Codigos presentes es TPM y no en la COOISPI
    "Codigos NA,Vacio, 0": df_Mezclas_nan_ceros,               #Valores nulos o ceros para gramaje, unidades producidas y promedio peso unidad
    "Errores": df_non_conver                                   #Valores no convertibles (NaN)
}
#Guardar el archivo de Novedades
fc.crear_archivo_novedades(Ruta_Novedades, dataframes_novedades)

Procesando Novedades...


✅ El Archivo de Novedades guardado exitosamente con 5 hojas guardadas correctamente.


In [110]:
df_non_conver.head()

,Número,Dia,Fecha,Mes:,IdMes,Año:,Máquina / Equipo:,Semana:,Turno:,Código y Descrip. / Producto:,Unidades Producidas (Conformes) :,Peso Promedio de la unidad (K):,Gramaje (K):
27255,84813,7,2025-08-07,Agosto,8,2025,INGEPACK,32,2,1003690 Cafe COLCAFE polvo ST1 Cj25tirilX24X10g,NaN,0.01006,0.01
